# Analyse politischer Trends

Meinungsforschungsinstitute betreiben einen beträchtlichen Aufwand, um die Meinungstrends der Bevölkerung bezogen auf Politiker mit Telefon- und Straßenumfragen zu erfassen. Die Meinungsforschung scheint jedoch in einer Krise zu stecken. Bereits beim Brexit waren einen Tag vor dem Referendum die meisten Meinungsforschungsinstitute zum Schluss gekommen, dass die Briten in der EU bleiben wollen. (http://www.zeit.de/politik/2016-06/umfragen-brexit-us-praesidentschaftswahl-demoskopie-einfluss) Aber auch bei der US-Wahl 2016 wurde in den meisten Fällen Clinton vorn gesehen.(http://www.morgenpost.de/politik/article208683451/Wie-Trumps-Wahlsieg-die-Demoskopen-in-die-Krise-stuerzt.html)

Die automatische Sentiment-Analyse könnte hier eine Möglichkeit sein, mit viel geringerem Aufwand aktuelle Meinungstrends zu erkennen und dafür Twitter-Daten zu analysieren.  Die Idee dahinter ist, dass die Plattform Twitter in Deutschland vielfach für politische Diskussionen genutzt wird.(In anderen Ländern ist z. B. auch Reddit sehr populär. Beispiele für Analysen von Reddit-Daten in Bezug auf Politiker sind Nithyanand et al. (2017) und Roozenbeek und Salvador (2017).) 
Eine Kernfrage der Untersuchung ist, ob Twitter als Quelle für Meinungsforschung dienen kann und klassische Meinungsforschung dadurch ersetzbar wird. Da sich Tweets auf einen Umfang von 140 Zeichen beschränken und das jeweilige Thema durch Hashtags meist eindeutig zugeordnet werden kann, scheinen sich Twitter-Daten gut für eine automatische Sentiment-Analyse zu eignen. Wir werden versuchen, diese Tweets automatisch in positive und negative Meinungsäußerungen zu klassifizieren. 
Anschließend versuchen wir, Tweets zu Politikern mit Veränderungen im ZDF-Politbarometer in Beziehung zu setzen. Damit nähern wir uns einer Antwort auf die Frage, ob man den Meinungstrend der Bevölkerung automatisch erfassen und damit letztlich klassische Meinungsforschung durch automatisierte Verfahren ersetzen kann.

Die Trendanalyse ist dabei natürlich nicht nur auf die Domäne der Politik und der Bewertung von Politikern beschränkt. Auch Firmen versuchen z. B. Trends zu erkennen und vorauszusagen, um ihr Produktportfolio und ihr Marketing darauf auszurichten. Man könnte z. B. auch versuchen, den Ausgang des Zuschauervotings für einen Gesangswettbewerb im Fernsehen anhand von Tweets vorauszusagen. Die hier vorgestellten Methoden sind auf viele weitere Anwendungsgebiete übertragbar.

## Aufstellung der Datenbasis

Für ein Experiment der Trendanalyse ist es zunächst notwendig, eine Datenbasis aufzustellen. In unserem Experiment sind das einerseits Tweets zu Politikerinnen und Politikern und andererseits Umfrageergebnisse der Meinungsforschung des ZDF-Politbarometers. 

### Tweets mit Meinungen zu Politikerinnen und Politikern

Twitter stellt für Forschungszwecke eine API zur Verfügung, die mit einer Python-Schnittstelle angesprochen werden kann. Damit lassen sich Tweets zu einem bestimmten Keyword oder Hashtag rückblickend für eine Woche aus der Twitter-Timeline extrahieren. Wir benötigen daher einen auf Python basierten Twitter-Crawler, der automatisch Tweets extrahiert, die sich mit einem der zehn Politiker beschäftigen, die auch im Politbarometer untersucht werden. Als erstes braucht man dafür einen Twitter-Account, der es erlaubt, über die API auf Tweets zuzugreifen. Mit diesem Account bekommt man Zugangstokens, die im Crawler-Programm eingefügt werden. Bei Twitter gibt es dafür eine Dokumentation unter https://developer.twitter.com/en/docs.
Außerdem brauchen wir die Python-Module TwitterSearch und urllib, die über pip install installiert und in Python importiert werden können. Hier ist der Code, mit dem mit einer Stichwortliste auf Twitter-Daten zugegriffen werden kann:


In [ ]:
# Import aller Module, die wir für den Twitter-Crawler benötigen

from urllib.request import urlopen

import TwitterSearch

from TwitterSearch import *
from pprint import pprint
import json

# Hier setzen wir die Keywords, nach denen gesucht wird!

keywords = ["Merkel", "Steinmeier"]

try:
    tso = TwitterSearchOrder() # Erstellen eines TwitterSearchOrder-Objekts
    tso.set_keywords(keywords) # Hier wird die Liste der Keywords aufgenommen
    tso.set_language('de') # Hier wird die Sprache gesetzt
    tso.set_include_entities(False) # wir wollen nicht die gesamte Information über alle Entities

# Das sind unsere geheimen Tokens, die wir von Twitter bekommen haben:
    ts = TwitterSearch(
        consumer_key = 'XXX',
        consumer_secret = 'YYY',
        access_token = 'xxx',
        access_token_secret = 'yyy'
         )

# Hier wird die Ausgabe erzeugt:
    for tweet in ts.search_tweets_iterable(tso):
        print( '\%s' \% tweet['created_at'] + '\t' +  @\%s tweeted: \%s' \% ( tweet['user']['screen_name'], tweet['text'] ) )

except TwitterSearchException as e:
    print(e)



Der nächste Schritt ist die Überlegung, welche Keywords eigentlich für die gesuchten PolitikerInnen relevant sind. So ist z. B. für eine Suche nach der Bundeskanzlerin Frau Merkel sicher auch das Keyword "Kanzlerin" relevant. Für die Aufstellung der relevanten Keywords ist es notwendig, in das aktuelle Twitter hinein zu sehen, mit einigen Keywords zu suchen und weitere aufzunehmen. 

Für eine Trendanalyse ist der zeitliche Aspekt relevant, denn man möchte ja die Entwicklung der Meinungen zu einem Thema analysieren. Deshalb haben wir die Zeitangabe der Tweets mit extrahiert und können die gewonnenen Daten in Dateien nach Datum aufteilen, die wir einzeln analysieren.

Im studentischen Projekt "Opinion Mining" (Siegel et al. 2017) haben wir mit dem Twitter-Crawler automatisch Tweets extrahiert, die sich mit den zehn Politikern beschäftigen, die zu diesem Zeitpunkt auch im Politbarometer untersucht wurden. In einer Voruntersuchung wurden die relevanten Keywords und Hashtags erfasst, mit denen der Twitter-Crawler dann gestartet wurde. Für jeden der zehn Politiker entstand so eine Textdatei, die alle Tweets der letzten sieben Tage enthält. Jede Zeile entspricht einem Tweet. Insgesamt wurden von jedem beteiligten Politiker über vier Wochen insgesamt 800 Tweets gesammelt, eine Gesamtmenge von 8.000 Tweets.

## ZDF-Politbarometer

Der Politbarometer wird von der Forschungsgruppe Wahlen e.V. (FGW) seit 1977 betrieben. Zu den Hauptaufgaben zählen die Betreuung und die wissenschaftliche Beratung der Sendungen des Zweiten Deutschen Fernsehens (ZDF).  Dabei werden Themenschwerpunkte wie politische Wahlen, Wählerverhalten, Meinungen zu politischen und gesellschaftlichen Fragen untersucht. Außerdem unterstützen und beraten sie bei der Verwendung von sozialwissenschaftlichen Daten. Für diese Untersuchung ist besonders der Aspekt "Beobachtungen gesellschaftlicher Trends und Stimmungen" interessant. Die Politbarometer-Untersuchungen ermitteln die zehn aktuell beliebtesten Spitzenpolitiker. Die Erhebung erfolgt telefonisch (dienstags bis donnerstags). Zielgruppe sind Wahlberechtigte in der Bundesrepublik Deutschland. Es werden in den westlichen Bundesländern ca. 1.000 zufällig ausgewählte Wahlberechtigte befragt, in den neuen Bundesländern 700. Um ein repräsentatives Ergebnis zu erhalten, werden die Befragungen im Osten überquotiert. Letztlich werden ca. 1.250 Interviews evaluiert. Weiterhin wird eine Zufallsauswahl der Befragten vorgenommen und nach mehreren Faktoren gewichtet. Die Bewertungsskala der Politiker beläuft sich zwischen -5 bis 5. (http://www.forschungsgruppe.de/Umfragen/Politbarometer/Methodik/)
Für den Vergleich mit Twitter nehmen wir die Beliebtheitswerte (und die Veränderung dieser Werte) von zwei Monaten für ausgewählte PolitikerInnen von der Webseite. Im Oktober und November des Jahres 2015 haben wir mit Studierenden der Hochschule Darmstadt ein solches Experiment durchgeführt. Abbildung 8.1 zeigt die Werte der Beliebtheitsskala für die untersuchten Politiker im November und Dezember 2015.

<img src="politbarometer.png">

## Sentiment-Analyse für Tweets
Im nächsten Schritt müssen die gesammelten Tweets daraufhin analysiert werden, ob sie Meinungsäußerungen enthalten und ob die Tweets positiv oder negativ sind. Für die Sentiment-Analyse haben wir ja bereits verschiedene Verfahren entwickelt, die hier zum Einsatz kommen können. Wenn die extrahierten Daten getrennt nach Politikern gesammelt wurden, dann ist in diesem Fall eine Sentiment-Analyse für Dokumente (Sentiment-Analyse auf Dokument-Ebene und nicht auf Satz-Ebene oder aspektbasiert) ausreichend. Um Trends zu erkennen, müssen die Tweets aber mit ihrem Datum gespeichert werden, denn die Untersuchung soll auch eine Veränderung über die Zeit beinhalten. 


## Zusammenfassung

In diesem Kapitel haben wir eine Anwendungsmöglichkeit für Sentiment-Analyse gezeigt, bei der politische Meinungstrends anhand von Twitter-Analysen vorausgesagt werden. Bei der Trendanalyse ist die Zeit des Social-Media-Beitrags eine wichtige Information. Wir haben gezeigt, wie Daten aus Twitter extrahiert werden können. 
Trendanalysen mithilfe von Sentiment-Analysen sind aber nicht auf politische Fragestellungen beschränkt. Sie werden z. B. für Börsenkurse (Minh et al. 2018) und Anforderungen an mobile Apps (Nagappan und Shihab 2016) gemacht. 
 
## Übungen
1. Prüfen Sie Ihr Wissen:
- Wie kann die Sentiment-Analyse zur Trendanalyse von politischen Diskussionen beitragen?
2. Setzen Sie Ihr neues Wissen ein:
- Erstellen Sie sich einen Account für die Twitter-API und nutzen Sie den Crawler, um nach Politikernamen zu suchen. 
- Erweitern Sie den Crawler so, dass die Ergebnisse in eine Datei geschrieben werden. 
- Suchen Sie aus dem ZDF-Politbarometer die aktuell untersuchten Politikernamen heraus und crawlen Sie Tweets zu diesen Politikern. Recherchieren Sie dazu die Keywords, die für die jeweiligen PolitikerInnen relevant sind. Diese sind neben den Namen auch Rollen, Schreibvarianten, Hashtags usw.
- Wenden Sie nun Ihre Sentiment-Analyse auf die extrahierten Tweets an und vergleichen Sie das Ergebnis mit dem Politbarometer. Können Sie eine Übereinstimmung feststellen? 
- Wiederholen Sie das Experiment mindestens noch einmal eine Woche später und analysieren Sie den Trend der Beliebtheit.  

3. Reflexion in Gruppenarbeit:
Diskutieren Sie in der Gruppe, ob ähnliche Sentiment-Analysen zur Trendanalyse verschiedener Social-Media-Daten auf Dauer die klassische Umfrage ersetzen könnte. Welche konkreten Chancen und welche Herausforderungen sehen Sie dabei?  Diskutieren Sie die Rolle der Sentiment-Analyse bei Trendanalysen in anderen Bereichen wie z. B. für die Erkennung von Produkttrends.



## Weiterführende Literatur
(Tumasjan et al. 2010) haben bereits 2009 versucht, mit der automatischen Analyse von Twitter die Ergebnisse der Bundestagswahl in Deutschland vorauszusagen. Dabei haben sie 100.000 Twitter-Nachrichten mit einer automatischen Text-Analyse-Software klassifiziert und die Ergebnisse mit den Ergebnissen der Bundestagswahl verglichen. Zu diesem Zeitpunkt war Twitter noch relativ neu, noch nicht mal vier Jahre alt. Die politische Diskussion in Twitter wurde von wenigen Nutzern beherrscht: "In sum, it becomes clear that, while Twitter is used as a forum for political deliberation, this forum is dominated by a small number of heavy users." (dt.: Insgesamt wird deutlich, dass Twitter von einer kleinen Anzahl sehr aktiver Nutzer dominiert wird, während es als Forum für politische Deliberation genutzt wird.(eigene Übersetzung)) (Tumasjan et al. 2010, S. 181).


Dennoch konnte eine klare Relation zwischen der Anzahl der Tweets zu einer politischen Partei und dem Wahlergebnis dieser Partei festgestellt werden: Je mehr Tweets zu einer Partei, desto besser das Ergebnis der Wahl. Die Abweichung war lediglich 1,65%, also nicht viel schlechter als bei den Vorhersagen der "Forschungsgruppe Wahlen" mit einer Abweichung von 1,48%. Jungherr et al. (2012) versuchten zwei Jahre später, diese Analyse nachzuvollziehen und kamen zum gegenteiligen Ergebnis: "The number of party mentions in the Twittersphere is thus not a valid indicator of offline political sentiment or even of future election outcomes." (dt.: Die Anzahl der Erwähnungen einer Partei in Twitter ist daher kein valider Indikator für Offline-Meinungen oder Wahlergebnissen.(eigene Übersetzung)) (Jungherr et al. 2012, S. 233).

Maynard und Funk (2011)  haben politische Tweets im Zusammenhang mit den britischen Parlamentswahlen im Jahr 2010 untersucht und automatisch Meinungsäußerungen klassifiziert. Im letzten Schritt haben sie aber  die Klassifikation der Tweets manuell korrigiert, denn die automatische Klassifikation hatte eine Präzision von 62,2%, was nicht aussagekräftig genug ist, um z. B. Wahlprognosen abzugeben.

